In [1]:
import numpy as np
import shutil
import os

from argparse import Namespace
from catboost import CatBoostRegressor, CatBoostClassifier

import sys
sys.path.append("../utils/")
sys.path.append("../calculating_predictions")
sys.path.append("../evaluation/")
from calculate_predictions import calculate_predictions
from evaluate import evaluate
from run_evaluation import *

%reload_ext autoreload
%autoreload 2

In [2]:
args = Namespace()
args.verbose = True
args.first_feature = 0
args.last_feature = -1
args.data_folder = "../../../data/best_features_days_data/"
out_folder = "substructed_target_predictions"
args.model_constructor=lambda verbose: CatBoostRegressor(verbose=False)
args.type = "regression"
args.metric_folder = "metrics"
args.additional_features = None
args.train_days = [3]
args.validation_day = None
args.test_days = [6]

def clear(folder):
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.mkdir(folder)

def get_stacking_metric(i):
    args.labels_to_substruct = {
        day: os.path.join("targets_to_substruct", str(i), "train_2_test_{}.npy".format(day))
        for day in [3, 6]
    }

    args.out_folder = os.path.join(out_folder, str(i))
    os.mkdir(args.out_folder)
    clear(args.metric_folder)    

    calculate_predictions(args)
    evaluate(args.out_folder, args.data_folder, args.metric_folder, argmax_positions)

    with open("metrics/metrics.txt") as handler:
        metric = float(next(handler).strip().split()[0])
        return metric

In [3]:
clear(out_folder)
metrics = [get_stacking_metric(i) for i in range(15)]

"2018-06-19 17:33:16.746003": preprocesing started
"2018-06-19 17:33:17.497297": train features shape: (20203, 80)
"2018-06-19 17:33:18.126685": preprocesing finished
"2018-06-19 17:33:18.126798": start training on days [3]
"2018-06-19 17:33:18.126898": using fit without validation
"2018-06-19 17:33:36.307111": built 1000 trees
"2018-06-19 17:33:38.382115": days to predict: [6]
"2018-06-19 17:33:38.382304": features shape: (247423, 80)
"2018-06-19 17:33:38.382421": start predicting on day 6
"2018-06-19 17:33:39.732653": predictions shape: (22493, 11)
"2018-06-19 17:33:39.732782": saveing results
"2018-06-19 17:33:39.734811": results saved
"2018-06-19 17:33:39.735306": predictions_filenames: ['substructed_target_predictions/0/train_3_test_6.npy']
"2018-06-19 17:33:39.735346": will evaluate days: [6]
"2018-06-19 17:33:39.735605": predict on file "substructed_target_predictions/0/train_3_test_6.npy"
"2018-06-19 17:33:41.801411": calculating metric
"2018-06-19 17:33:41.802207": preprocesin

"2018-06-19 17:37:07.691833": calculating metric
"2018-06-19 17:37:07.692546": preprocesing started
"2018-06-19 17:37:08.511529": train features shape: (20203, 80)
"2018-06-19 17:37:09.159936": preprocesing finished
"2018-06-19 17:37:09.160007": start training on days [3]
"2018-06-19 17:37:09.160104": using fit without validation
"2018-06-19 17:37:28.395660": built 1000 trees
"2018-06-19 17:37:30.443422": days to predict: [6]
"2018-06-19 17:37:30.443510": features shape: (247423, 80)
"2018-06-19 17:37:30.443528": start predicting on day 6
"2018-06-19 17:37:31.967418": predictions shape: (22493, 11)
"2018-06-19 17:37:31.967576": saveing results
"2018-06-19 17:37:31.969285": results saved
"2018-06-19 17:37:31.969707": predictions_filenames: ['substructed_target_predictions/9/train_3_test_6.npy']
"2018-06-19 17:37:31.969742": will evaluate days: [6]
"2018-06-19 17:37:31.969865": predict on file "substructed_target_predictions/9/train_3_test_6.npy"
"2018-06-19 17:37:33.991482": calculating

In [5]:
np.mean(metrics), np.std(metrics) / np.sqrt(15)

(0.6238528456084617, 0.0038283226717096674)

In [6]:
metrics

[0.595910113962814,
 0.6311024947211112,
 0.6095383948995858,
 0.617252493213872,
 0.6286519710106868,
 0.6314363624958749,
 0.6324636739243606,
 0.6048832970381357,
 0.6308729534412181,
 0.6104813137573333,
 0.6477041265474871,
 0.6284633703441627,
 0.6209895561093888,
 0.6157449844280264,
 0.652297578232868]

In [7]:
x = np.load("substructed_target_predictions/0/train_3_test_6.npy")

In [8]:
x

array([[ 0.00462072,  0.00462072,  0.00508601, ...,  0.00521781,
         0.00397825, -0.00105554],
       [ 0.01309353,  0.01309353,  0.01283806, ...,  0.00576462,
         0.00730773,  0.01059598],
       [ 0.02326003,  0.02326003,  0.02300456, ...,  0.02234412,
         0.0196867 ,  0.01785485],
       ...,
       [ 0.03508375,  0.03508375,  0.03569043, ...,  0.0385282 ,
         0.04029139,  0.04357963],
       [ 0.08075507,  0.08075507,  0.0804996 , ...,  0.07560534,
         0.07827175,  0.078878  ],
       [-0.05007032, -0.05007032, -0.04946364, ..., -0.05123515,
        -0.05425978, -0.05358347]])